# PySpark Transformations and SQL

In [1]:
from pathlib import Path

import delta
import pyspark
import pyspark.sql.functions as F
from delta import configure_spark_with_delta_pip

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.executor.memory", "10G")
    .config("spark.driver.memory", "25G")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.sql.shuffle.partitions", "2")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-350-delta-310/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-514772ca-8ebd-4811-b0c2-3990660a9428;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 109ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default 

In [3]:
import quinn

In [27]:
df = (
    spark.createDataFrame(
        [("bob", 30, "canada"), ("carol", 45, "us"), ("sue", 5, "canada")]
    )
    .toDF("First Name", "Person Age", "Country Of Origin")
    .select("Country Of Origin", "Person Age", "First Name")
)

In [28]:
df.show()

+-----------------+----------+----------+
|Country Of Origin|Person Age|First Name|
+-----------------+----------+----------+
|           canada|        30|       bob|
|               us|        45|     carol|
|           canada|         5|       sue|
+-----------------+----------+----------+



In [29]:
df = df.transform(quinn.snake_case_col_names)
spark.sql("select * from {df} order by all", df=df).show()

+-----------------+----------+----------+
|country_of_origin|person_age|first_name|
+-----------------+----------+----------+
|           canada|         5|       sue|
|           canada|        30|       bob|
|               us|        45|     carol|
+-----------------+----------+----------+

